In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML 

In [6]:
df = pd.read_csv("raw_results/xgb_p_h_0.0/results_0.csv")
pd.crosstab(df.A, df.B)

B,0,1
A,,
0,3639,599
1,947,4657


In [7]:
df = pd.read_csv("raw_results/bert_p_h_rho_0p00_dgp_seed_109_random_state_14_results.csv")
pd.crosstab(df.A, df.B)

B,0,1
A,,
0,1278,212
1,598,7754


In [10]:
(np.sqrt(9842)*(0.006503))/0.257302

2.5073344765142633

In [11]:
(np.sqrt(9842)*(0.006503))/0.305231

2.1136194406075166

In [9]:
(np.sqrt(9842)*(0.002540))/0.305231

0.8255564169065189

In [12]:
(np.sqrt(9842)*(0.002540))/0.257302

0.9793371629011577